# NDArray: mutidimensional SChunk

NDArray functions let users perform different operations with NDArray arrays like setting, copying or slicing them.
In this section, we are going to see how to create and manipulate a NDArray array in a simple way.


In [1]:
import blosc2
import numpy as np

## Creating an array
First, we create an array, with zeros being used as the default value for uninitialized portions of the array.


In [2]:
array = blosc2.zeros((10000, 10000), dtype=np.int32)
print(array.info)

type    : NDArray
shape   : (10000, 10000)
chunks  : (512, 1024)
blocks  : (128, 256)
dtype   : int32
cratio  : 65536.00
cparams : {'blocksize': 131072,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 4,
 'use_dict': 0}
dparams : {'nthreads': 2}



Note that all the compression and decompression parameters, as well as the chunks and blocks shapes are set to the default.

## Reading and writing data
We can access and edit NDArray arrays using NumPy.

In [3]:
array[0, :] = np.arange(10000, dtype=array.dtype)
array[:, 0] = np.arange(10000, dtype=array.dtype)

In [4]:
array[0, 0]

array(0, dtype=int32)

In [5]:
array[0, :]

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int32)

In [6]:
array[:, 0]

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int32)

In [7]:
array[...]

array([[   0,    1,    2, ..., 9997, 9998, 9999],
       [   1,    0,    0, ...,    0,    0,    0],
       [   2,    0,    0, ...,    0,    0,    0],
       ...,
       [9997,    0,    0, ...,    0,    0,    0],
       [9998,    0,    0, ...,    0,    0,    0],
       [9999,    0,    0, ...,    0,    0,    0]], dtype=int32)

## Persistent data
As in the SChunk, when we create a NDArray array, we can specify where it will be stored. Indeed, we can specify all the compression/decompression parameters that we can specify in a SChunk.
So as in the SChunk, to store an array on-disk we only have to specify a `urlpath` where to store the new array.


In [8]:
array = blosc2.full((1000, 1000), fill_value=b"pepe", chunks=(100, 100), blocks=(50, 50),
             urlpath="ndarray_tutorial.b2nd", mode="w")
print(array.info)

type    : NDArray
shape   : (1000, 1000)
chunks  : (100, 100)
blocks  : (50, 50)
dtype   : |S4
cratio  : 1111.11
cparams : {'blocksize': 10000,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 4,
 'use_dict': 0}
dparams : {'nthreads': 2}



This time we even set the chunks and blocks shapes. You can now open it with modes `w`, `a` or `r`.

In [9]:
array2 = blosc2.open("ndarray_tutorial.b2nd")
print(array2.info)

type    : NDArray
shape   : (1000, 1000)
chunks  : (100, 100)
blocks  : (50, 50)
dtype   : |S4
cratio  : 1111.11
cparams : {'blocksize': 10000,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 4,
 'use_dict': 0}
dparams : {'nthreads': 2}



## Compression params
Here we can see how when we make a copy of a NDArray array we can change its compression parameters in an easy way.

In [10]:
b = np.arange(1000000).tobytes()
array1 = blosc2.frombuffer(b, shape=(1000, 1000), dtype=np.int64, chunks=(500, 10), blocks=(50, 10))
print(array1.info)

type    : NDArray
shape   : (1000, 1000)
chunks  : (500, 10)
blocks  : (50, 10)
dtype   : int64
cratio  : 7.45
cparams : {'blocksize': 4000,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 8,
 'use_dict': 0}
dparams : {'nthreads': 2}



In [11]:
cparams = {
    "codec": blosc2.Codec.ZSTD,
    "clevel": 9,
    "filters": [blosc2.Filter.BITSHUFFLE],
    "filters_meta": [0]
}

array2 = array1.copy(chunks=(500, 10), blocks=(50, 10), cparams=cparams)
print(array2.info)

type    : NDArray
shape   : (1000, 1000)
chunks  : (500, 10)
blocks  : (50, 10)
dtype   : int64
cratio  : 13.94
cparams : {'blocksize': 4000,
 'clevel': 9,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.BITSHUFFLE: 2>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 8,
 'use_dict': 0}
dparams : {'nthreads': 2}



## Metalayers and variable length metalayers

We have seen that you can pass to the NDArray constructor any compression or decompression parameters that you may pass to a SChunk. Indeed, you can also pass the metalayer dict. Metalayers are small metadata for informing about the properties of data that is stored on a container. As explained in [the SChunk basics](00.schunk-basics.html), there are two kinds. The first one (`meta`), cannot be deleted, must be added at construction time and can only be updated with values that have the same bytes size as the old value. They are easy to access and edit by users:

In [12]:
meta = {
    "dtype": "i8",
    "coords": [5.14, 23.]
}
array = blosc2.zeros((1000, 1000), dtype=np.int16, chunks=(100, 100), blocks=(50, 50), meta=meta)

You can work with them like if you were working with a dictionary. To access this dictionary you will use the SChunk attribute that an NDArray has.

In [13]:
array.schunk.meta

In [14]:
array.schunk.meta.keys()

['b2nd', 'dtype', 'coords']

As you can see, Blosc2 internally uses these metalayers to store shapes, ndim, dtype, etc, and retrieve this data when needed in the `b2nd` metalayer.

In [15]:
array.schunk.meta["b2nd"]

[0, 2, [1000, 1000], [100, 100], [50, 50], 0, '<i2']

In [16]:
array.schunk.meta["coords"]

[5.14, 23.0]

To add a metalayer after the creation or a variable length metalayer, you can use the `vlmeta` accessor from the SChunk. As well as the `meta`, it works similarly to a dictionary.

In [17]:
print(array.schunk.vlmeta.getall())
array.schunk.vlmeta["info1"] = "This is an example"
array.schunk.vlmeta["info2"] = "of user meta handling"
array.schunk.vlmeta.getall()

{}


{b'info1': 'This is an example', b'info2': 'of user meta handling'}

You can update them with a value larger than the original one:

In [18]:
array.schunk.vlmeta["info1"] = "This is a larger example"
array.schunk.vlmeta.getall()

{b'info1': 'This is a larger example', b'info2': 'of user meta handling'}

## Creating a NDArray from a NumPy array

Let's create a NDArray from a NumPy array using the `asarray` constructor:

In [19]:
shape = (100, 100, 100)
dtype = np.float64
nparray = np.linspace(0, 100, np.prod(shape), dtype=dtype).reshape(shape)
b2ndarray = blosc2.asarray(nparray)
print(b2ndarray.info)

type    : NDArray
shape   : (100, 100, 100)
chunks  : (100, 100, 100)
blocks  : (32, 32, 32)
dtype   : float64
cratio  : 16.00
cparams : {'blocksize': 262144,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 8,
 'use_dict': 0}
dparams : {'nthreads': 2}



## Building a NDArray from a buffer

Furthermore, you can create a NDArray filled with data from a buffer:

In [20]:
buffer = bytes(np.random.normal(0, 1, np.prod(shape)) * 8)
b2ndarray = blosc2.frombuffer(buffer, shape, dtype=dtype)
print("Compression ratio:", b2ndarray.schunk.cratio)
b2ndarray[:5, :5, :5]

Compression ratio: 2.3465074245437014


array([[[ -4.43078053,  -9.23140176,  -1.42970124,   7.86492952,
           5.96161416],
        [ -0.90332728,  14.4083556 ,  -0.19161072,  -6.78668455,
          -5.36322617],
        [ -1.2499399 ,   8.10738718,  14.07369165,   4.04316805,
           0.76039683],
        [ -1.66511563,   2.7451846 ,   3.31973828,  -1.01945568,
          -2.45495568],
        [  6.5559866 ,  -7.74970461,   5.35510224, -10.37733392,
           0.86824383]],

       [[  7.91110565,  -0.40447467,   7.88094809,   3.83898103,
          11.58337252],
        [  1.04113739,  10.72055765,  -0.5534428 ,   7.91836773,
           6.02385272],
        [  1.99058377,  -3.16799092,  10.99784718,  -6.20992068,
          12.91831185],
        [ 10.53330643, -21.32174973,  -3.44512813,   9.83905792,
           4.70180599],
        [  0.08870868,  -9.94890713,  -9.38701242,  -3.90103209,
           1.5765455 ]],

       [[  2.44583632,  -6.66355678,  -2.37807004, -12.04591163,
           4.22702233],
        [ 12.6918

That's all for now.  There are more examples in the [examples directory of the git repository](https://github.com/Blosc/python-blosc2/tree/main/examples/) for you to explore.  Enjoy!